# Static Fracture Analysis (SFA) with Python
## ***Instructions to use this notebook***
### 1. Install Anaconda

- Install [anaconda for **Python 3.6**](https://www.anaconda.com/download/) for your OS
- Accept all defaults and add Anaconda to your Path when prompted

### 2. Create environment

It is important to work in a defined environment and to ensure all packages are installed and run in that environement, to that end run the following commands in a terminal, in your working directory for this notebook:

`conda config --add channels conda-forge` This 'channel' will now always be available in your environments

`conda create -n fracs python=3.6` To create an environment called 'fracs' using Python3.6

`conda activate fracs` To activate the 'fracs' environment (which you can leave with `conda deactivate`)

### 3. Install packages to environment

While still in the 'fracs' environment , install the following packages (other packages can be installed later):

`conda install -c conda-forge pandas matplotlib numpy ipython`

`pip install xlrd`

### 4. Enable environment in jupyter notebooks and launch

`python -m ipykernel install --name fracs --display-name "fracs"` To enable the environment

`jupyter notebook` To launch the notebook in a browser

Then create a new notebook using the 'fracs' kernel

### 5. Library imports

In [56]:
# these warning filters are needed for a numpy error
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed") # due a RuntimeWarning with numpy.dtype
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from utils import mdia_to_xyz_minCurve
from mpl_toolkits.mplot3d import Axes3D
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### 6. Data import and cleaning

In [57]:
# import data
# well survey
survey = pd.read_csv('../data/HEF_image_interp/for_HEF/Well_Surveys_Projected_to_TD.csv', header=4)
# well tops
tops = pd.read_excel('../data/HEF_image_interp/for_HEF/Well_Formations.xlsx')
# image data
image_data = pd.read_csv('../data/HEF_image_interp/ASCII_n_LAS/well_1D_all.txt',skiprows=(1,2))

In [58]:
# clean up survey
# drop NaN
survey_edt = survey.dropna(how='all')
survey_edt = survey_edt.dropna(axis='columns', how='all')
survey_edt.head()

,MD[m],Inc[deg],Azi[deg],TVD[m],North[m],East[m],Dogleg [deg/30m],Vertical Section[m]
0,0,0.0,0.00,0.00,0.00,0.00,NaN,0.00
1,76.29,0.9,7.19,76.29,0.59,0.07,0.35,0.26
2,104.36,0.9,17.20,104.35,1.02,0.17,0.17,0.41
3,134,0.9,15.59,133.99,1.47,0.30,0.03,0.53
4,162.73,1.1,6.39,162.72,1.96,0.39,0.27,0.72


In [59]:
survey_edt.tail(2)

,MD[m],Inc[deg],Azi[deg],TVD[m],North[m],East[m],Dogleg [deg/30m],Vertical Section[m]
79,2267.00,35.43,298.39,2013.3,498.84,-797.35,0.29,940.51
82,All data is in Metres unless otherwise stated\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# Copy survey information from survey dataframe and save to seperate dataframe
survey_info = survey_edt[-1:]
survey_info = survey_info.drop(columns=list(survey_edt)[1:],inplace=False)
survey_info.rename(index=str, columns={"MD[m]": "Survey Info"}, inplace=True)
survey_info.reset_index(inplace=True, drop=True)
survey_info

,Survey Info
0,All data is in Metres unless otherwise stated\...


In [61]:
# Delete Survey information from survey dataframe
survey_edt = survey_edt[:-1]
survey_edt.tail()

,MD[m],Inc[deg],Azi[deg],TVD[m],North[m],East[m],Dogleg [deg/30m],Vertical Section[m]
75,2171.08,35.90,298.69,1935.50,472.03,-748.06,0.33,884.53
76,2199.36,36.00,298.69,1958.40,480.00,-762.62,0.11,901.10
77,2228.15,35.80,298.39,1981.72,488.07,-777.45,0.28,917.10
78,2248.73,35.80,298.49,1998.43,493.79,-788.01,0.30,929.93
79,2267.00,35.43,298.39,2013.30,498.84,-797.35,0.29,940.51


In [62]:
# convert data to floats
survey_edt = survey_edt.apply(pd.to_numeric)

In [63]:
tops.head(3)

,DD-11 Formations,Unnamed: 1,Unnamed: 2,Unnamed: 3,YMT Comments
0,MD,TVD,Formations,Log Tops,Remarks
1,5.5,5.5,Lower Bakhtiari,5.5,No Gamma
2,445,441.37,Upper Fars,445,No Gamma


In [64]:
# Clean up tops
tops.columns = tops.iloc[0]
tops.drop(columns=['Log Tops', 'Remarks'], inplace=True)
tops.drop([0], inplace=True)
tops.reset_index(inplace=True, drop=True)
tops = tops.apply(pd.to_numeric, errors='ignore')
tops

,MD,TVD,Formations
0,5.50,5.50,Lower Bakhtiari
1,445.00,441.37,Upper Fars
2,810.00,770.91,Lower Fars
3,1150.00,1065.27,Pila Spi
4,1318.00,1210.50,Gercus
5,1372.00,1257.41,Sinjar
6,1495.00,1364.37,Kolosh
7,1885.00,1699.87,Shiranish
8,2032.00,1822.94,Kometan
9,2049.01,1836.73,Upper Qamchuqa


In [65]:
image_data.head()

,#DEPTH DIP AZIM QUAL DEVI HAZI AAZI ADIP DIPTYPE
0,1886.85 5.5 321.8 1.00 30.9 262.4 27...
1,1887.40 12.9 329.4 1.00 30.8 257.0 27...
2,1887.97 6.7 326.7 1.00 30.9 255.0 26...
3,1888.53 10.5 320.5 1.00 30.8 263.3 27...
4,1889.34 6.7 336.3 1.00 30.8 298.0 30...


In [66]:
headers = image_data.columns.values

In [67]:
headers = headers[0].split()
headers

['#DEPTH', 'DIP', 'AZIM', 'QUAL', 'DEVI', 'HAZI', 'AAZI', 'ADIP', 'DIPTYPE']

In [68]:
image_data = image_data['#DEPTH  DIP    AZIM   QUAL   DEVI   HAZI   AAZI   ADIP      DIPTYPE'].str.split(n=None,expand=True)

In [69]:
image_data.head()

,0,1,2,3,4,5,6,7,8,9
0,1886.85,5.5,321.8,1.00,30.9,262.4,270.8,34.0,BED,None
1,1887.40,12.9,329.4,1.00,30.8,257.0,277.8,36.6,BED,None
2,1887.97,6.7,326.7,1.00,30.9,255.0,266.5,33.5,BED,None
3,1888.53,10.5,320.5,1.00,30.8,263.3,277.8,37.4,BED,None
4,1889.34,6.7,336.3,1.00,30.8,298.0,305.0,36.3,BED,None


In [70]:
image_data.fillna(value='',inplace=True)
image_data.head()

,0,1,2,3,4,5,6,7,8,9
0,1886.85,5.5,321.8,1.00,30.9,262.4,270.8,34.0,BED,
1,1887.40,12.9,329.4,1.00,30.8,257.0,277.8,36.6,BED,
2,1887.97,6.7,326.7,1.00,30.9,255.0,266.5,33.5,BED,
3,1888.53,10.5,320.5,1.00,30.8,263.3,277.8,37.4,BED,
4,1889.34,6.7,336.3,1.00,30.8,298.0,305.0,36.3,BED,


In [71]:
image_data[8] = image_data[8] + image_data[9]
image_data.drop(columns=[9], inplace=True)
image_data.columns = headers
image_data = image_data.apply(pd.to_numeric, errors='ignore')
image_data.head()

,#DEPTH,DIP,AZIM,QUAL,DEVI,HAZI,AAZI,ADIP,DIPTYPE
0,1886.85,5.5,321.8,1.0,30.9,262.4,270.8,34.0,BED
1,1887.40,12.9,329.4,1.0,30.8,257.0,277.8,36.6,BED
2,1887.97,6.7,326.7,1.0,30.9,255.0,266.5,33.5,BED
3,1888.53,10.5,320.5,1.0,30.8,263.3,277.8,37.4,BED
4,1889.34,6.7,336.3,1.0,30.8,298.0,305.0,36.3,BED


### 7. Cleaned data export for backup

In [72]:
# export cleaned data to csv
survey_edt.to_csv('../data/cleanedData/survey_edt.csv',sep=',')
survey_info.to_csv('../data/cleanedData/survey_info.csv',sep=',')
tops.to_csv('../data/cleanedData/tops.csv',sep=',')
image_data.to_csv('../data/cleanedData/image_data.csv',sep=',')

### 8. Exploratory data analysis

#### 8.1 Well deviation survey

In [73]:
#survey_edt, survey_info, tops, image_data
survey_edt[39:42]

,MD[m],Inc[deg],Azi[deg],TVD[m],North[m],East[m],Dogleg [deg/30m],Vertical Section[m]
39,1143.36,30.3,297.79,1059.55,205.14,-283.44,0.75,349.23
40,1171.49,30.3,297.79,1083.83,211.76,-296.00,0.00,363.38
41,1199.59,30.5,297.69,1108.07,218.38,-308.58,0.22,377.55


In [74]:
survey_info['Survey Info'][0]

"All data is in Metres unless otherwise stated\r\nCoordinates are from Installation MD's are from Rig and TVD's are from Rig ( Actual Datum #1 430.1m above Mean Sea Level )\r\nVertical Section is from 0.00N 0.00E on azimuth 302.380 degrees\r\nBottom hole distance is 901.11 Metres on azimuth 302.19 degrees from Wellhead\r\nCalculation method uses Minimum Curvature method"

In [75]:
def mdia_to_xyz_minCurve(deviation):
    """
    A function to convert a well deviation (path of csv file) given in MD[m], incl[deg], azi[deg]
    into an xyz array with x[m], y[m], z[m] using the minimum curvature method
    according to: [drillingformulas.com](http://bit.ly/2MNp7U0)
    """
    # read data
    data = pd.read_csv(deviation, sep=',', header='infer')
    # clean data
    data.drop(columns=['Unnamed: 0'], inplace=True)
    # add columns needed for calculations
    data['Dogleg_rad [rad/30m]'] = np.radians(data['Dogleg [deg/30m]'])
    data['RatioFactor'] = (2 / data['Dogleg_rad [rad/30m]']) * np.tan(data['Dogleg_rad [rad/30m]'] / 2)
    print(data['RatioFactor'][39:42])
    data['RatioFactor'].replace(np.nan, 1, inplace=True)
    print(data['RatioFactor'][39:42])
    # calculate intervals
    delta_MD = np.array(data['MD[m]'][1:]) - np.array(data['MD[m]'][:-1])
    # get uppers/lowers
    RF_lower = np.array(data['RatioFactor'][1:])
    incl_upper = np.array(data['Inc[deg]'][:-1])
    incl_lower = np.array(data['Inc[deg]'][1:])
    azi_upper = np.array(data['Azi[deg]'][:-1])
    azi_lower = np.array(data['Azi[deg]'][1:])
    # calculate xyz
    east_x = delta_MD / 2 * (np.sin(np.radians(incl_upper)) * np.sin(np.radians(azi_upper)) 
                             + np.sin(np.radians(incl_lower)) * np.sin(np.radians(azi_lower))) * RF_lower
    
    north_y = delta_MD / 2 * (np.sin(np.radians(incl_upper)) * np.cos(np.radians(azi_upper)) 
                             + np.sin(np.radians(incl_lower)) * np.cos(np.radians(azi_lower))) * RF_lower
    
    TVD_z = delta_MD / 2 * (np.cos(np.radians(incl_upper)) + np.cos(np.radians(incl_lower))) * RF_lower
    
    return east_x, north_y, TVD_z


In [80]:
# get xyz arrays for survey
x_dev = mdia_to_xyz_minCurve('../data/cleanedData/survey_edt.csv')[0]
y_dev = mdia_to_xyz_minCurve('../data/cleanedData/survey_edt.csv')[1]
z_dev = mdia_to_xyz_minCurve('../data/cleanedData/survey_edt.csv')[2]

39    1.000014
40         NaN
41    1.000001
Name: RatioFactor, dtype: float64
39    1.000014
40    1.000000
41    1.000001
Name: RatioFactor, dtype: float64
39    1.000014
40         NaN
41    1.000001
Name: RatioFactor, dtype: float64
39    1.000014
40    1.000000
41    1.000001
Name: RatioFactor, dtype: float64
39    1.000014
40         NaN
41    1.000001
Name: RatioFactor, dtype: float64
39    1.000014
40    1.000000
41    1.000001
Name: RatioFactor, dtype: float64


In [21]:
x_pos = []
x_temp = 0
for line in x_dev:
    x_temp += line
    x_pos.append(x_temp)

y_pos = []    
y_temp = 0
for line in y_dev:
    y_temp += line
    y_pos.append(y_temp)

z_pos = []
z_temp = 0
for line in z_dev:
    z_temp += line
    z_pos.append(-1 * z_temp)

In [22]:
survey_edt.head(3)

,MD[m],Inc[deg],Azi[deg],TVD[m],North[m],East[m],Dogleg [deg/30m],Vertical Section[m]
0,0.00,0.0,0.00,0.00,0.00,0.00,NaN,0.00
1,76.29,0.9,7.19,76.29,0.59,0.07,0.35,0.26
2,104.36,0.9,17.20,104.35,1.02,0.17,0.17,0.41


In [23]:
# add np.nan value at 0-index to get equal length with survey
x_pos.insert(0,np.nan)
y_pos.insert(0,np.nan)
z_pos.insert(0,np.nan)

In [24]:
# add new column to df
survey_edt['x[m]'] = x_pos
survey_edt['y[m]'] = y_pos
survey_edt['z[m]'] = z_pos
survey_edt.head()

,MD[m],Inc[deg],Azi[deg],TVD[m],North[m],East[m],Dogleg [deg/30m],Vertical Section[m],x[m],y[m],z[m]
0,0.00,0.0,0.00,0.00,0.00,0.00,NaN,0.00,NaN,NaN,NaN
1,76.29,0.9,7.19,76.29,0.59,0.07,0.35,0.26,0.074991,0.594446,-76.285531
2,104.36,0.9,17.20,104.35,1.02,0.17,0.17,0.41,0.167772,1.023758,-104.352089
3,134.00,0.9,15.59,133.99,1.47,0.30,0.03,0.53,0.299168,1.470349,-133.988433
4,162.73,1.1,6.39,162.72,1.96,0.39,0.27,0.72,0.390501,1.961742,-162.714067


In [25]:
survey_edt[38:42]

,MD[m],Inc[deg],Azi[deg],TVD[m],North[m],East[m],Dogleg [deg/30m],Vertical Section[m],x[m],y[m],z[m]
38,1114.51,29.9,298.99,1034.59,198.26,-270.71,0.77,334.80,-270.714105,198.263451,-1034.591116
39,1143.36,30.3,297.79,1059.55,205.14,-283.44,0.75,349.23,-283.442416,205.141716,-1059.550939
40,1171.49,30.3,297.79,1083.83,211.76,-296.00,0.00,363.38,NaN,NaN,NaN
41,1199.59,30.5,297.69,1108.07,218.38,-308.58,0.22,377.55,NaN,NaN,NaN


In [26]:
(2/np.radians(0.000001)) * np.tan(np.radians(0.000001)/2)

1.0

In [27]:
@interact(azimuth=widgets.IntSlider(min=0,max=360,step=5,value=270)
          , elevation=widgets.IntSlider(min=0,max=90,step=5,value=90))
def rotate(azimuth, elevation):
    """
    displays the well trajectory in 3D
    default view is from top with North at top
    """
    # set up 
    mpl.rcParams['legend.fontsize'] = 12
    fig = plt.figure(figsize=(10,10))
    ax = fig.gca(projection='3d')

    # pass data
    x = x_pos
    y = y_pos
    z = z_pos

    # define plot
    ax.plot(x, y, z, label='Well deviation')
    ax.legend(loc=0)
    ax.view_init(elev = elevation, azim=azimuth)
    plt.show()
    return

interactive(children=(IntSlider(value=270, description='azimuth', max=360, step=5), IntSlider(value=90, descri…

#### 8.2 Well tops

In [58]:
tops

,MD,TVD,Formations
0,5.50,5.50,Lower Bakhtiari
1,445.00,441.37,Upper Fars
2,810.00,770.91,Lower Fars
3,1150.00,1065.27,Pila Spi
4,1318.00,1210.50,Gercus
5,1372.00,1257.41,Sinjar
6,1495.00,1364.37,Kolosh
7,1885.00,1699.87,Shiranish
8,2032.00,1822.94,Kometan
9,2049.01,1836.73,Upper Qamchuqa


In [59]:
survey_edt.head()

,MD[m],Inc[deg],Azi[deg],TVD[m],North[m],East[m],Dogleg [deg/30m],Vertical Section[m],x[m],y[m],z[m]
0,0.00,0.0,0.00,0.00,0.00,0.00,NaN,0.00,NaN,NaN,NaN
1,76.29,0.9,7.19,76.29,0.59,0.07,0.35,0.26,0.074991,0.594446,-76.285531
2,104.36,0.9,17.20,104.35,1.02,0.17,0.17,0.41,0.167772,1.023758,-104.352089
3,134.00,0.9,15.59,133.99,1.47,0.30,0.03,0.53,0.299168,1.470349,-133.988433
4,162.73,1.1,6.39,162.72,1.96,0.39,0.27,0.72,0.390501,1.961742,-162.714067


In [60]:
def top_incAzi(tops_df):
    """
    Takes a tops dataframe and returns same dataframe with addtional columns
    for Inc and Azi based on:
    inc = inc_dev_lower - ((MD_dev_lower - MD_Top) * (delta_incl) / (delta_MD_dev))
    azi = azi_dev_lower - ((MD_dev_lower - MD_Top) * (delta_azi) / (delta_MD_dev))
    """
    for top in tops_df.itertuples():
        current_Top_MD = top[1] 
        for depth in survey_edt.itertuples():
            current_dev_MD = depth[1]
            if current_Top_MD < current_dev_MD:
                # get row above and below top
                enclosing_rows = survey_edt[depth[0] - 1:depth[0] + 1]
                # grab MD, incl, azi values from rows
                MD_dev_low = enclosing_rows[1:2]['MD[m]']
                MD_dev_up = enclosing_rows[:1]['MD[m]']
                Inc_dev_low = enclosing_rows[1:2]['Inc[deg]']
                Inc_dev_up = enclosing_rows[:1]['Inc[deg]']
                Azi_dev_low = enclosing_rows[1:2]['Azi[deg]']
                Azi_dev_up = enclosing_rows[:1]['Azi[deg]']
                # grab x,y,z from rows
                x_dev_low = enclosing_rows[1:2]['x[m]']
                x_dev_up = enclosing_rows[:1]['x[m]']
                y_dev_low = enclosing_rows[1:2]['y[m]']
                y_dev_up = enclosing_rows[:1]['y[m]']
                z_dev_low = enclosing_rows[1:2]['z[m]']
                z_dev_up = enclosing_rows[:1]['z[m]']
                # assign calculated value to correct column of correct row
                # use .loc to avoid warning about assigning to a copy
                tops_df.loc[top[:1][0],'Inc[deg]'] = Inc_dev_low.values[0] - ((MD_dev_low.values[0] - current_Top_MD) 
                                                                * (Inc_dev_low.values[0] - Inc_dev_up.values[0]) 
                                                                / (MD_dev_low.values[0] - MD_dev_up.values[0]))
                tops_df.loc[top[:1][0],'Azi[deg]'] = Azi_dev_low.values[0] - ((MD_dev_low.values[0] - current_Top_MD) 
                                                                * (Azi_dev_low.values[0] - Azi_dev_up.values[0]) 
                                                                / (MD_dev_low.values[0] - MD_dev_up.values[0]))
                
                tops_df.loc[top[:1][0],'x[m]'] = x_dev_low.values[0] - ((MD_dev_low.values[0] - current_Top_MD) 
                                                                * (x_dev_low.values[0] - x_dev_up.values[0]) 
                                                                / (MD_dev_low.values[0] - MD_dev_up.values[0]))
                tops_df.loc[top[:1][0],'y[m]'] = y_dev_low.values[0] - ((MD_dev_low.values[0] - current_Top_MD) 
                                                                * (y_dev_low.values[0] - y_dev_up.values[0]) 
                                                                / (MD_dev_low.values[0] - MD_dev_up.values[0]))
                tops_df.loc[top[:1][0],'z[m]'] = z_dev_low.values[0] - ((MD_dev_low.values[0] - current_Top_MD) 
                                                                * (z_dev_low.values[0] - z_dev_up.values[0]) 
                                                                / (MD_dev_low.values[0] - MD_dev_up.values[0]))
                break
    return tops_df

In [61]:
tops = top_incAzi(tops)
tops

,MD,TVD,Formations,Inc[deg],Azi[deg],x[m],y[m],z[m]
0,5.50,5.50,Lower Bakhtiari,0.064884,0.518351,NaN,NaN,NaN
1,445.00,441.37,Upper Fars,14.782877,324.611918,-11.565505,38.615367,-441.374055
2,810.00,770.91,Lower Fars,29.908803,303.971338,-139.890420,120.326525,-770.918182
3,1150.00,1065.27,Pila Spi,30.300000,297.790000,NaN,NaN,NaN
4,1318.00,1210.50,Gercus,29.791073,298.481073,NaN,NaN,NaN
5,1372.00,1257.41,Sinjar,29.604817,298.390000,NaN,NaN,NaN
6,1495.00,1364.37,Kolosh,29.311869,299.629836,NaN,NaN,NaN
7,1885.00,1699.87,Shiranish,31.435415,301.506681,NaN,NaN,NaN
8,2032.00,1822.94,Kometan,35.625881,299.270589,NaN,NaN,NaN
9,2049.01,1836.73,Upper Qamchuqa,35.889500,299.072875,NaN,NaN,NaN


In [29]:
survey_edt[38:42]

,MD[m],Inc[deg],Azi[deg],TVD[m],North[m],East[m],Dogleg [deg/30m],Vertical Section[m],x[m],y[m],z[m]
38,1114.51,29.9,298.99,1034.59,198.26,-270.71,0.77,334.80,-270.714105,198.263451,-1034.591116
39,1143.36,30.3,297.79,1059.55,205.14,-283.44,0.75,349.23,-283.442416,205.141716,-1059.550939
40,1171.49,30.3,297.79,1083.83,211.76,-296.00,0.00,363.38,NaN,NaN,NaN
41,1199.59,30.5,297.69,1108.07,218.38,-308.58,0.22,377.55,NaN,NaN,NaN


#### 8.3 Fracture data

### 9. Static Fracture Analysis

<hr>

![agile_logo](agile_logo_size_50.jpg)
<p style="text-align:center;color:gray">© Agile Geoscience 2018. Licensed CC-BY.</p>
</hr>